In [40]:
from mwrogue.esports_client import EsportsClient
from mwrogue.auth_credentials import AuthCredentials

credentials = AuthCredentials(user_file="me2")
site = EsportsClient('lol', credentials=credentials)

"SG.Team1Dragons, " \
            "SG.Team2Dragons,
             "SG.Team1Barons," \
            "SG.Team2Barons,""SG.Team1RiftHeralds," \
            "SG.Team2RiftHeralds," \
            "SG.Team1VoidGrubs," \
            "SG.Team2VoidGrubs," \
            "SG.Team1Atakhans" \
            "SG.Team2Atakhans"

In [41]:
# Pegando os dados da Leagupedia
response = site.cargo_client.query(
    tables = "ScoreboardGames=SG",
    fields="SG.Tournament, " \
            "SG.Team1, " \
            "SG.Team2, " \
            "SG.WinTeam, " \
            "SG.LossTeam, " \
            "SG.Team1Picks, " \
            "SG.Team2Picks, " \
            "SG.Team1Players, " \
            "SG.Team2Players, " \
            "SG.Team1Dragons, " \
            "SG.Team2Dragons, " \
            "SG.Team1Clouds, " \
            "SG.Team1Infernals, " \
            "SG.Team1Mountains, " \
            "SG.Team1Oceans, " \
            "SG.Team1Hextechs, " \
            "SG.Team1Chemtechs, " \
            "SG.Team1Elders, " \
            "SG.Team2Clouds, " \
            "SG.Team2Infernals, " \
            "SG.Team2Mountains," \
            "SG.Team2Oceans," \
            "SG.Team2Hextechs," \
            "SG.Team2Chemtechs," \
            "SG.Team2Elders," \
            "SG.Team1Barons," \
            "SG.Team2Barons," \
            "SG.Team1Towers," \
            "SG.Team2Towers," \
            "SG.Team1Gold," \
            "SG.Team2Gold," \
            "SG.Team1Kills," \
            "SG.Team2Kills," \
            "SG.Team1RiftHeralds," \
            "SG.Team2RiftHeralds," \
            "SG.Team1VoidGrubs," \
            "SG.Team2VoidGrubs," \
            "SG.Team1Atakhans," \
            "SG.Team2Atakhans",
    where='SG.Tournament = "IDL Kings Lendas Season 3"'
)

# Validando as informações
for rows in response:
    for item, value in rows.items():
        print(item, value)

Tournament IDL Kings Lendas Season 3
Team1 100Vices
Team2 ÉanDG
WinTeam ÉanDG
LossTeam 100Vices
Team1Picks Yorick,Skarner,Viktor,Zeri,Neeko
Team2Picks K'Sante,Pantheon,Hwei,Sivir,Karma
Team1Players Pijack,sarolu,Takeshi,Celo,Cavalo
Team2Players zynts,randal,Qats,brTT,Telas
Team1Dragons 2
Team2Dragons 4
Team1Clouds 0
Team1Infernals 0
Team1Mountains 1
Team1Oceans 0
Team1Hextechs 1
Team1Chemtechs 0
Team1Elders 0
Team2Clouds 0
Team2Infernals 0
Team2Mountains 3
Team2Oceans 0
Team2Hextechs 0
Team2Chemtechs 1
Team2Elders 0
Team1Barons 0
Team2Barons 2
Team1Towers 5
Team2Towers 10
Team1Gold 69069
Team2Gold 76822
Team1Kills 15
Team2Kills 38
Team1RiftHeralds 0
Team2RiftHeralds 1
Team1VoidGrubs 3
Team2VoidGrubs 0
Team1Atakhans 0
Team2Atakhans 1
Tournament IDL Kings Lendas Season 3
Team1 100Vices
Team2 FONatic
WinTeam FONatic
LossTeam 100Vices
Team1Picks Sion,Jarvan IV,Ryze,Xayah,Rakan
Team2Picks Ambessa,Wukong,Taliyah,Zeri,Alistar
Team1Players Pijack,sarolu,Takeshi,Celo,Cavalo
Team2Players zekas,D

In [57]:
# Importando o pandas e mantendo os dados brutos em um DataFrame para não ficar importando os dados via API o tempo inteiro
# Há um limite para uso da API

import pandas as pd

dataframe = pd.DataFrame(response)

static_dataframe = dataframe

In [58]:
# Criando o nome para as novas colunas do DataFrame
# Elas serão geradas da separação das colunas de Picks e Players

cols_1 = ["Team1_top","Team1_jg","Team1_mid","Team1_adc","Team1_sup"]
cols_2 = ["Team2_top","Team2_jg","Team2_mid","Team2_adc","Team2_sup"]
cols_3 = ["Team1player_top","Team1player_jg", "Team1player_mid","Team1player_adc","Team1player_sup"]
cols_4 = ["Team2player_top","Team2player_jg", "Team2player_mid","Team2player_adc","Team2player_sup"]

In [59]:
# Separando os dados de pick e players dos 2 times para análise detalhada por players e campeões
dataframe[cols_1] = dataframe["Team1Picks"].str.split(",", expand=True, n=4)
dataframe[cols_2] = dataframe["Team2Picks"].str.split(",", expand=True, n=4)
dataframe[cols_3] = dataframe["Team1Players"].str.split(",", expand=True, n=4)
dataframe[cols_4] = dataframe["Team2Players"].str.split(",", expand=True, n=4)

# Droppando as colunas que foram separadas (Não são mais úteis)
dataframe = dataframe.drop(columns=["Team1Picks","Team2Picks","Team1Players","Team2Players"])

In [60]:
# Aplicando uma função lambda de ordenação nas colunas referentes à posição jg no dataframe
# É necessário ordenar para poder realizar a contagem de quantas vezes determinada MatchUp aconteceu
jg_matchup = dataframe[["Team1_jg","Team2_jg"]].apply(lambda s: tuple(sorted(s)), axis=1)

# Contando a quantidade de vezes que uma determinada matchup aconteceu
jg_matchup.value_counts()

(Wukong, Xin Zhao)       8
(Jarvan IV, Wukong)      7
(Pantheon, Xin Zhao)     7
(Sejuani, Xin Zhao)      5
(Jarvan IV, Xin Zhao)    5
(Pantheon, Skarner)      5
(Skarner, Wukong)        4
(Qiyana, Trundle)        4
(Nocturne, Trundle)      4
(Jarvan IV, Pantheon)    4
(Jarvan IV, Trundle)     4
(Sejuani, Vi)            4
(Trundle, Wukong)        4
(Jax, Xin Zhao)          3
(Nocturne, Xin Zhao)     3
(Jarvan IV, Nocturne)    3
(Sejuani, Wukong)        3
(Skarner, Vi)            2
(Trundle, Xin Zhao)      2
(Maokai, Xin Zhao)       2
(Vi, Xin Zhao)           2
(Trundle, Vi)            2
(Vi, Wukong)             2
(Nocturne, Pantheon)     2
(Pantheon, Trundle)      2
(Jarvan IV, Skarner)     2
(Jarvan IV, Sejuani)     2
(Poppy, Skarner)         1
(Dr. Mundo, Skarner)     1
(Nocturne, Qiyana)       1
(Pantheon, Vi)           1
(Jarvan IV, Qiyana)      1
(Nocturne, Sejuani)      1
(Jax, Nocturne)          1
(Naafiri, Trundle)       1
(Lillia, Zyra)           1
(Nocturne, Skarner)      1
(

In [50]:
champ1, champ2 = "Wukong", "Xin Zhao"

df_jg_mask = ((dataframe["Team1_jg"].eq(champ1) & dataframe["Team2_jg"].eq(champ2)) | (dataframe["Team1_jg"].eq(champ2) & dataframe["Team2_jg"].eq(champ1)))

most_jg_matchup_dataframe = dataframe[df_jg_mask]

In [53]:
most_jg_matchup_dataframe[["Team1","Team2","WinTeam","Team1_jg", "Team2_jg", "Team1player_jg", "Team2player_jg","Team1Dragons","Team2Dragons","Team1Barons","Team2Barons","Team1RiftHeralds","Team2RiftHeralds","Team1VoidGrubs","Team2VoidGrubs","Team1Atakhans","Team2Atakhans"]]

,Team1,Team2,WinTeam,Team1_jg,Team2_jg,Team1player_jg,Team2player_jg,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1RiftHeralds,Team2RiftHeralds,Team1VoidGrubs,Team2VoidGrubs,Team1Atakhans,Team2Atakhans
23,FONatic,G12,G12,Wukong,Xin Zhao,Drakehero,Minerva,2,2,0,2,1,0,3,0,0,1
45,G12,Tepei Esports,G12,Xin Zhao,Wukong,Minerva,Dizin,2,2,1,0,1,0,2,1,1,0
61,Gen GG,Tepei Esports,Tepei Esports,Wukong,Xin Zhao,SamKz,Dizin,0,5,0,1,1,0,1,2,0,1
68,Karmine Cospe,FONatic,FONatic,Wukong,Xin Zhao,Shini,Drakehero,1,5,0,1,1,0,2,1,0,1
74,Karmine Cospe,Tepei Esports,Karmine Cospe,Wukong,Xin Zhao,Shini,Dizin,1,3,1,0,1,0,0,3,0,1
76,Karmine Cospe,Vôs Grandes,Karmine Cospe,Wukong,Xin Zhao,Shini,stiner,2,1,1,0,1,0,1,2,1,0
99,paiNtriotas,Tepei Esports,paiNtriotas,Wukong,Xin Zhao,Accez,Dizin,2,3,0,1,0,1,3,0,1,0
108,Tepei Esports,Oreiudos Esports,Tepei Esports,Xin Zhao,Wukong,Dizin,Aegis (Gabriel Lemos),2,3,0,0,1,0,3,0,1,0


In [52]:
dataframe[["Team1_jg","Team2_jg"]].stack().value_counts()

Xin Zhao     39
Wukong       32
Jarvan IV    29
Trundle      26
Pantheon     24
Sejuani      17
Skarner      17
Vi           17
Nocturne     17
Qiyana        7
Jax           5
Graves        4
Maokai        3
Naafiri       2
Poppy         2
Zyra          2
Dr. Mundo     1
Lillia        1
Lee Sin       1
Ivern         1
Zed           1
Name: count, dtype: int64